In this lab we will examining how to chain outputs from one prompt to the next.

In [ ]:
from operator import itemgetter

from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.llms.bedrock import Bedrock
import boto3

REGION_NAME="us-east-1" ## change to your region
PROFILE_NAME="lza-comm-gss"  ## change to your desired aws credential profile
## ensure Anthropic Claude is enabled in your AWS Account.
named_profile = boto3.session.Session(profile_name=PROFILE_NAME)
bedrock_client = named_profile.client('bedrock-runtime')
print('Initalizing Anthropic Claude v2')
model = Bedrock(
    client=bedrock_client,
    model_id="anthropic.claude-v2",
    endpoint_url="https://bedrock-runtime." + REGION_NAME + ".amazonaws.com",
    model_kwargs={"temperature": 0}
)

LLMs often need specific instructions and structure in order to reason through a task.  The more complex the task, the more structure it needs. 

### Chains
In this task we will be asking Bedrock - given a type of food, what country is famous for that food? what is the capital of that country (which requires the output from first question) and what is a recipe for that food (recalling the original input)?

Notice the two separate prompts one to answer the first part - the country, which will pass down to the second prompt. 

[We use LCEL to construct the chain1 for the first prompt.](https://python.langchain.com/docs/expression_language/get_started)

In the second chain, notice the second prompt has two inputs - country and food - so we must pass them in a dictionary - country must call the output of the first chain, and then we must use the item getter to get the original input.

### Model differences
Notice we must define a model for each chain - it's important to know that different models are better at different tasks - like text generation vs image generation.  We will explore this in a different lab.  In this lab, we will use Anthropic for both.

We then invoke chain 2 (which calls chain1), and pass it a dictionary with food as the key and whatever food we want to test this with as the value.

In [ ]:


prompt1 = ChatPromptTemplate.from_template("what country is famous for {food} ?")
prompt2 = ChatPromptTemplate.from_template(
    "what the capital city of {country} in? provide a recipe for {food}"
)
chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"country": chain1, "food": itemgetter("food")}
    | prompt2
    | model
    | StrOutputParser()
)

print(chain2.invoke({"food": "pizza"}))